In [1]:
import numpy as np
import pandas as pd

In [107]:
train = pd.read_csv('E:/kaggle/Benz/data/train.csv/train.csv')
test = pd.read_csv('E:/kaggle/Benz/data/test.csv/test.csv')

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [17]:
len(train)

4209

In [15]:
test['y'] = -1

In [16]:
test.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X376,X377,X378,X379,X380,X382,X383,X384,X385,y
0,1,az,v,n,f,d,t,a,w,0,...,0,0,1,0,0,0,0,0,0,-1
1,2,t,b,ai,a,d,b,g,y,0,...,0,1,0,0,0,0,0,0,0,-1
2,3,az,v,as,f,d,a,j,j,0,...,0,0,1,0,0,0,0,0,0,-1
3,4,az,l,n,f,d,z,l,n,0,...,0,0,1,0,0,0,0,0,0,-1
4,5,w,s,as,c,d,y,i,m,0,...,0,0,0,0,0,0,0,0,0,-1


In [22]:
train_test = pd.concat((train, test), axis=0)

C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [23]:
train_test

,ID,X0,X1,X10,X100,X101,X102,X103,X104,X105,...,X91,X92,X93,X94,X95,X96,X97,X98,X99,y
0,0,k,v,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,130.81
1,6,k,t,0,1,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,88.53
2,7,az,w,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,76.26
3,9,az,t,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,80.62
4,13,az,v,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,78.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8410,aj,h,0,1,1,0,1,0,0,...,0,0,0,0,0,1,0,1,0,-1.00
4205,8411,t,aa,0,1,1,0,1,0,0,...,0,0,0,0,0,1,0,1,0,-1.00
4206,8413,y,v,0,1,1,0,1,0,0,...,0,0,0,0,0,1,0,1,0,-1.00
4207,8414,ak,v,0,1,1,0,1,0,0,...,0,0,0,0,0,1,0,1,0,-1.00


## LabelEncoder to transfer categorical features to numerical ones
- Here I write a new class in case that I maybe put this operation to Pipeline

In [7]:
class LabelEncoderTransfomer(BaseEstimator, TransformerMixin):
    def fit(self, X, y = None):
        """
        X : Dataframe, which needed to be transformed
        """
        return self
    def transform(self, X, y = None):
        for column_name in X.columns:
            if X[column_name].dtype == 'object' and (column_name != 'y'):
                le = LabelEncoder()
                standardscaler = StandardScaler()
                X[column_name] = le.fit_transform(X[column_name])
                X[column_name] = pd.Series(standardscaler.fit_transform(X[column_name].values.reshape(-1,1)).reshape(-1))
        return X

In [26]:
label_transform = LabelEncoderTransfomer()
label_transform.fit(train_test)
new_train_test = label_transform.transform(train_test)

In [30]:
new_train_test.to_csv('new_train_test.csv')

# Here We Try Some Demension-Reduction Methods

In [31]:
from sklearn.decomposition import PCA, TruncatedSVD, FastICA

## tSVD

In [33]:
tsvd = TruncatedSVD(n_components = 12)
tsvd_train_test = tsvd.fit_transform(new_train_test.drop('y', axis = 1))

In [34]:
df_tsvd_train_test = pd.DataFrame(tsvd_train_test)

In [35]:
df_tsvd_train_test

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.004187,3.504640,0.448356,2.540893,-1.092674,-1.035575,0.549141,-0.276696,0.033261,-0.431983,1.064780,-0.146923
1,6.006371,5.370430,-0.443093,-0.313193,-1.554533,-0.099392,0.195953,-0.887304,-0.286928,-0.971999,1.422245,-0.277710
2,7.005382,4.362660,-1.510002,2.528752,-1.989034,1.989749,1.151217,-1.736666,3.776413,0.687438,0.233074,-0.544641
3,9.004889,3.850197,-1.369152,1.399486,-1.985495,3.359381,1.009941,-2.251635,1.865824,-0.167175,-1.194269,-0.580179
4,13.004808,4.206323,-1.500189,1.181247,-2.244818,3.689727,0.508670,-1.479261,2.326718,1.797962,-1.171942,-0.452691
...,...,...,...,...,...,...,...,...,...,...,...,...
8413,8410.002106,-1.825387,-3.113885,-0.422523,0.556627,0.545228,0.563886,1.108247,0.343997,1.140874,0.215526,-0.067482
8414,8411.002024,-2.042546,1.864371,1.016534,-1.416667,-0.170655,2.106344,-1.303946,-0.785766,0.618829,-0.319423,0.081116
8415,8413.001446,-2.652414,-1.482261,-1.255950,1.223946,2.059268,1.719786,-0.713328,-1.179701,0.468298,-0.402466,0.490422
8416,8414.002613,-1.219981,-2.514477,1.285928,0.878828,0.120052,0.691112,-0.458032,0.239292,-0.499400,2.087022,-0.360091


# PCA

In [37]:
pca = PCA(n_components = 12)
pca_train_test = pca.fit_transform(new_train_test.drop('y', axis = 1))

In [38]:
df_pca_train_test = pd.DataFrame(pca_train_test)

In [39]:
df_pca_train_test

,0,1,2,3,4,5,6,7,8,9,10,11
0,-4208.499477,0.532109,2.217775,-1.805524,0.337378,1.345693,-0.196151,0.298937,-0.457754,0.689447,0.153159,-0.410301
1,-4202.499309,-0.510979,-0.504593,-1.376521,-0.445898,0.100867,-0.893468,-0.037290,-0.993333,1.228246,0.056109,-0.568656
2,-4201.499344,-1.434948,1.982969,-2.964873,2.070493,-0.266574,-1.879875,3.252355,1.685636,0.848694,-0.672356,0.168887
3,-4199.499351,-1.227089,0.778288,-3.043404,3.023373,-1.268789,-2.476929,1.767112,0.318589,-0.564533,-1.010763,1.056174
4,-4195.499833,-1.383928,0.551971,-3.189972,2.900028,-1.816926,-1.660214,1.783393,2.307281,-0.895818,-0.589956,-0.503373
...,...,...,...,...,...,...,...,...,...,...,...,...
8413,4201.499435,-3.133986,-0.342656,0.587272,0.362604,-0.007551,0.791541,-0.396450,1.352632,0.619742,-0.268626,0.683773
8414,4202.499391,1.865696,0.855904,-1.377175,-0.361784,1.247382,-1.870090,-1.188893,0.508367,-0.125714,-0.021292,-0.106625
8415,4204.499529,-1.407667,-1.213379,0.937728,2.106543,-0.214216,-1.349363,-1.649294,0.302366,-0.106015,0.205222,0.737280
8416,4205.499353,-2.620561,1.490403,0.905006,-0.271381,0.084113,-0.758810,-0.169294,-0.196169,2.492780,-0.261001,0.341145


## FastICA

In [40]:
fast_ica = FastICA(n_components = 12)
fast_ica_train_test = fast_ica.fit_transform(new_train_test.drop('y',axis = 1))

In [41]:
df_fast_ica = pd.DataFrame(fast_ica_train_test)

In [42]:
df_fast_ica

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.005359,0.001447,-0.008220,-0.019460,0.007316,-0.001310,-0.007725,0.019330,-0.001171,-0.003421,0.007916,-0.008493
1,0.007390,0.004519,0.005224,0.003056,0.007611,0.002938,-0.006691,0.019987,-0.006135,-0.000954,0.009616,-0.012737
2,0.008008,0.003004,-0.009886,-0.008736,0.018228,-0.047304,-0.007116,0.018602,-0.009920,-0.007538,-0.001868,-0.008228
3,0.004792,0.005980,-0.001771,-0.002306,-0.010362,-0.046511,0.000152,0.020364,-0.009750,-0.002591,0.003020,-0.013588
4,-0.001363,0.003588,-0.001049,-0.001889,0.002614,-0.050076,-0.002897,0.020431,-0.013328,0.007369,0.005006,0.009682
...,...,...,...,...,...,...,...,...,...,...,...,...
8413,0.006165,-0.006024,0.003843,0.000720,0.001750,-0.000519,0.005160,-0.019406,-0.017623,-0.001167,-0.016362,0.008488
8414,-0.009227,-0.000324,-0.005056,-0.006340,-0.009161,0.000139,-0.021775,-0.017749,-0.007416,-0.006929,0.013491,-0.002757
8415,-0.011514,-0.007044,0.007584,0.005345,-0.018206,-0.002579,-0.001985,-0.016885,-0.018461,0.008569,-0.008793,-0.007707
8416,0.012613,-0.000411,-0.011504,0.004420,0.009686,0.006707,-0.004782,-0.018632,-0.020061,-0.000259,-0.011258,-0.016233


# Append decomposition components to datasets

In [46]:
df = pd.DataFrame()
for i in range(12):
    df['tsvd_' + str(i)] = df_tsvd_train_test.iloc[:, i]
    df['pca_' + str(i)] = df_pca_train_test.iloc[:, i]
    df['fast_ica_' + str(i)] = df_fast_ica.iloc[:, i]

In [47]:
df

,tsvd_0,pca_0,fast_ica_0,tsvd_1,pca_1,fast_ica_1,tsvd_2,pca_2,fast_ica_2,tsvd_3,...,fast_ica_8,tsvd_9,pca_9,fast_ica_9,tsvd_10,pca_10,fast_ica_10,tsvd_11,pca_11,fast_ica_11
0,0.004187,-4208.499477,0.005359,3.504640,0.532109,0.001447,0.448356,2.217775,-0.008220,2.540893,...,-0.001171,-0.431983,0.689447,-0.003421,1.064780,0.153159,0.007916,-0.146923,-0.410301,-0.008493
1,6.006371,-4202.499309,0.007390,5.370430,-0.510979,0.004519,-0.443093,-0.504593,0.005224,-0.313193,...,-0.006135,-0.971999,1.228246,-0.000954,1.422245,0.056109,0.009616,-0.277710,-0.568656,-0.012737
2,7.005382,-4201.499344,0.008008,4.362660,-1.434948,0.003004,-1.510002,1.982969,-0.009886,2.528752,...,-0.009920,0.687438,0.848694,-0.007538,0.233074,-0.672356,-0.001868,-0.544641,0.168887,-0.008228
3,9.004889,-4199.499351,0.004792,3.850197,-1.227089,0.005980,-1.369152,0.778288,-0.001771,1.399486,...,-0.009750,-0.167175,-0.564533,-0.002591,-1.194269,-1.010763,0.003020,-0.580179,1.056174,-0.013588
4,13.004808,-4195.499833,-0.001363,4.206323,-1.383928,0.003588,-1.500189,0.551971,-0.001049,1.181247,...,-0.013328,1.797962,-0.895818,0.007369,-1.171942,-0.589956,0.005006,-0.452691,-0.503373,0.009682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8413,8410.002106,4201.499435,0.006165,-1.825387,-3.133986,-0.006024,-3.113885,-0.342656,0.003843,-0.422523,...,-0.017623,1.140874,0.619742,-0.001167,0.215526,-0.268626,-0.016362,-0.067482,0.683773,0.008488
8414,8411.002024,4202.499391,-0.009227,-2.042546,1.865696,-0.000324,1.864371,0.855904,-0.005056,1.016534,...,-0.007416,0.618829,-0.125714,-0.006929,-0.319423,-0.021292,0.013491,0.081116,-0.106625,-0.002757
8415,8413.001446,4204.499529,-0.011514,-2.652414,-1.407667,-0.007044,-1.482261,-1.213379,0.007584,-1.255950,...,-0.018461,0.468298,-0.106015,0.008569,-0.402466,0.205222,-0.008793,0.490422,0.737280,-0.007707
8416,8414.002613,4205.499353,0.012613,-1.219981,-2.620561,-0.000411,-2.514477,1.490403,-0.011504,1.285928,...,-0.020061,-0.499400,2.492780,-0.000259,2.087022,-0.261001,-0.011258,-0.360091,0.341145,-0.016233


# Here we get the data for our model

## 1. Data for stacked model

In [49]:
label_transform.fit(train.drop('y', axis=1))
X_train_stacked = label_transform.transform(train.drop('y', axis=1))

In [71]:
X_train_stacked = X_train_stacked.drop('ID', axis = 1)

In [51]:
y_train_stacked = train['y']

In [88]:
label_transform.fit(test)
test_stacked = label_transform.transform(test)

In [90]:
test_stacked.drop('ID', axis = 1 ,inplace = True)

In [101]:
test_stacked.drop('y', axis = 1, inplace = True)

## 2. Data for XGB

In [52]:
X_train_xgb = df.iloc[:4209, :]

In [54]:
y_train_xgb = train['y']

# Model

In [55]:
import xgboost as xgb
from xgboost import XGBClassifier

In [56]:
xgb_params = {
    'n_trees': 520, 
    'eta': 0.0045,
    'max_depth': 4,
    'subsample': 0.93,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_train_xgb.mean(), # base prediction = mean(target)
    'silent': 1
}

In [57]:
dtrain = xgb.DMatrix(X_train_xgb, y_train_xgb)
model = xgb.train(xgb_params, dtrain, num_boost_round = 1250)

C:\Users\lenovo\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\lenovo\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [59]:
xgb_predict = model.predict(dtrain)

In [60]:
from sklearn.metrics import r2_score

In [61]:
xgb_evaluation = r2_score(y_train_xgb, xgb_predict)

In [62]:
xgb_evaluation

0.5987383574444116

# Stacked Model

In [63]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LassoLarsCV, LassoCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.utils import check_array

In [64]:
class StackingEstimator(BaseEstimator, TransformerMixin):
    def __init__(self, estimator):
        self.estimator = estimator
    
    def fit(self, X, y = None):
        self.estimator.fit(X, y)
        return self
    
    def transform(self, X, y = None):
        X = check_array(X)
        X_transformed = np.copy(X)
        X_transformed = np.hstack((X, self.estimator.predict(X).reshape(-1,1)))
        return X_transformed

In [72]:
estimator = GradientBoostingRegressor(learning_rate=0.001, loss="huber", max_depth=6, max_features = 'auto', min_samples_leaf=18, min_samples_split=14, subsample=0.7)
stackingestimator = StackingEstimator(estimator)
stackingestimator.fit(X_train_stacked, y_train_stacked)
X_transformed = stackingestimator.transform(X_train_stacked)

In [76]:
stacked_pipeline = Pipeline(
    [('LassoLarsCV', StackingEstimator(LassoLarsCV(cv = 5))),
     ('GradientBoostingRegressor', StackingEstimator(GradientBoostingRegressor(learning_rate=0.001, loss="huber", max_depth=6, max_features = 'auto', min_samples_leaf=18, min_samples_split=14, subsample=0.7))),
     ('LassoLarsCV2', LassoLarsCV())]
)

In [77]:
stacked_pipeline.fit(X_train_stacked, y_train_stacked)

C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=5.294e-02, with an active set of 2 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=4.985e-02, with an active set of 3 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=4.985e-02, with an active set of 3 regressors, and the smallest cholesky pi

C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 17 iterations, i.e. alpha=1.297e-02, with an active set of 17 regressors, and the smallest cholesky pivot element being 1.054e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 21 iterations, i.e. alpha=1.116e-02, with an active set of 21 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 23 iterations, i.e. alpha=1.039e-02, with an active set of 23 regressors, and the smallest chole

C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 5 iterations, i.e. alpha=6.362e-03, with an active set of 5 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 10 iterations, i.e. alpha=4.563e-03, with an active set of 10 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
C:\Users\lenov

Pipeline(memory=None,
         steps=[('LassoLarsCV',
                 StackingEstimator(estimator=LassoLarsCV(copy_X=True, cv=5,
                                                         eps=2.220446049250313e-16,
                                                         fit_intercept=True,
                                                         max_iter=500,
                                                         max_n_alphas=1000,
                                                         n_jobs=None,
                                                         normalize=True,
                                                         positive=False,
                                                         precompute='auto',
                                                         verbose=False))),
                ('GradientBoostingRegressor',
                 StackingEstimator(estimator=GradientBoostingRegressor(alpha=0.9,
                                                                   

In [95]:
stacked_prediction = stacked_pipeline.predict(X_train_stacked)

In [94]:
X_train_stacked.shape

(4209, 376)

In [96]:
stacked_evaluation = r2_score(y_train_stacked, stacked_prediction)

In [97]:
stacked_evaluation

0.5950045220080578

# Make predictions on test set

In [85]:
test_xgb = df.iloc[4209:, :]
dtest = xgb.DMatrix(test_xgb)

In [86]:
test_predict_xgb = model.predict(dtest)

In [102]:
test_predict_stacked = stacked_pipeline.predict(test_stacked)

In [112]:
final_prediction = 0.75 * test_predict_xgb + 0.25 * test_predict_stacked

In [118]:
test['y'] = pd.Series(test_predict_stacked)

In [119]:
test[['ID', 'y']].to_csv('my_submission2.csv')